# Named entities recognition (on Twitter) using LSTM

*based on NLP course on coursera*
Named Entity Recognition (NER) is a common task in natural language processing.<br>
It serves for extraction such entities from the text as persons, organizations, locations, etc. 


In [1]:
import sys
sys.path.append("..")
from common.download_utils import download_week2_resources

In [2]:
download_week2_resources()

HBox(children=(IntProgress(value=0, max=849548), HTML(value='')))

HBox(children=(IntProgress(value=0, max=103771), HTML(value='')))

HBox(children=(IntProgress(value=0, max=106837), HTML(value='')))

What we need is a corpus that contains twits with named entity tags.Every line of a file contains a pair of a token (word/punctuation symbol) and a tag, separated by a whitespace. Different tweets are separated by an empty line.

#### Loading data 

In [3]:
def read_data(file_path):
    tokens = []
    tags = []
    
    tweet_tokens = []
    tweet_tags = []
    for line in open(file_path, encoding='utf-8'):
        line = line.strip()
        if not line:
            if tweet_tokens:
                tokens.append(tweet_tokens)
                tags.append(tweet_tags)
            tweet_tokens = []
            tweet_tags = []
        else:
            token, tag = line.split()
            if token.startswith('@'):
                token = '<USR>'
            elif token.startswith('http://') or token.startswith('https://'):
                token = '<URL>'
            tweet_tokens.append(token)
            tweet_tags.append(tag)
            
    return tokens, tags

In [4]:
train_tokens, train_tags = read_data('data/train.txt')
validation_tokens, validation_tags = read_data('data/validation.txt')
test_tokens, test_tags = read_data('data/test.txt')

Learn the data:

In [6]:
for i in range(3):
    for token, tag in zip(train_tokens[i], train_tags[i]):
        print('%s\t%s' % (token, tag))
    print()

RT	O
<USR>	O
:	O
Online	O
ticket	O
sales	O
for	O
Ghostland	B-musicartist
Observatory	I-musicartist
extended	O
until	O
6	O
PM	O
EST	O
due	O
to	O
high	O
demand	O
.	O
Get	O
them	O
before	O
they	O
sell	O
out	O
...	O

Apple	B-product
MacBook	I-product
Pro	I-product
A1278	I-product
13.3	I-product
"	I-product
Laptop	I-product
-	I-product
MD101LL/A	I-product
(	O
June	O
,	O
2012	O
)	O
-	O
Full	O
read	O
by	O
eBay	B-company
<URL>	O
<URL>	O

Happy	O
Birthday	O
<USR>	O
!	O
May	O
Allah	B-person
s.w.t	O
bless	O
you	O
with	O
goodness	O
and	O
happiness	O
.	O



### Dictionaries
for tokens and tags


In [8]:
from collections import defaultdict

In [9]:
def build_dict(tokens_or_tags, special_tokens):
    """
        tokens_or_tags: a list of lists of tokens or tags
        special_tokens: some special tokens
    """
    # a dictionary with default value 0
    tok2idx = defaultdict(lambda: 0)
    idx2tok = []

    idx = 0
    for token in special_tokens:
        idx2tok.append(token)
        tok2idx[token] = idx
        idx += 1

    for token_or_tag in tokens_or_tags:
        for token in token_or_tag:
            if token not in tok2idx:
                idx2tok.append(token)
                tok2idx[token] = idx
                idx += 1
    
    return tok2idx, idx2tok

In [13]:
special_tokens = ['<UNK>', '<PAD>']
special_tags = ['O']

Special tokens:
- <UNK> token for out of vocabulary tokens;
- <PAD> token for padding sentence to the same length when we create batches of sentences.

In [14]:
# Creating dictionaries 
token2idx, idx2token = build_dict(train_tokens + validation_tokens, special_tokens)
tag2idx, idx2tag = build_dict(train_tags, special_tags)

In [18]:
idx2tag

['O',
 'B-musicartist',
 'I-musicartist',
 'B-product',
 'I-product',
 'B-company',
 'B-person',
 'B-other',
 'I-other',
 'B-facility',
 'I-facility',
 'B-sportsteam',
 'B-geo-loc',
 'I-geo-loc',
 'I-company',
 'I-person',
 'B-movie',
 'I-movie',
 'B-tvshow',
 'I-tvshow',
 'I-sportsteam']

Functions for mapping between tokens and ids for a sentence

In [19]:
def words2idxs(tokens_list):
    return [token2idx[word] for word in tokens_list]

def tags2idxs(tags_list):
    return [tag2idx[tag] for tag in tags_list]

def idxs2words(idxs):
    return [idx2token[idx] for idx in idxs]

def idxs2tags(idxs):
    return [idx2tag[idx] for idx in idxs]

#### Batches 
= pad/truncate the to short/to long sequences

In [20]:
def batches_generator(batch_size, tokens, tags,
                      shuffle=True, allow_smaller_last_batch=True):
    """Generates padded batches of tokens and tags."""
    
    n_samples = len(tokens)
    if shuffle:
        order = np.random.permutation(n_samples)
    else:
        order = np.arange(n_samples)

    n_batches = n_samples // batch_size
    if allow_smaller_last_batch and n_samples % batch_size:
        n_batches += 1

    for k in range(n_batches):
        batch_start = k * batch_size
        batch_end = min((k + 1) * batch_size, n_samples)
        current_batch_size = batch_end - batch_start
        x_list = []
        y_list = []
        max_len_token = 0
        for idx in order[batch_start: batch_end]:
            x_list.append(words2idxs(tokens[idx]))
            y_list.append(tags2idxs(tags[idx]))
            max_len_token = max(max_len_token, len(tags[idx]))
            
        # Fill in the data into numpy nd-arrays filled with padding indices.
        x = np.ones([current_batch_size, max_len_token], dtype=np.int32) * token2idx['<PAD>']
        y = np.ones([current_batch_size, max_len_token], dtype=np.int32) * tag2idx['O']
        lengths = np.zeros(current_batch_size, dtype=np.int32)
        for n in range(current_batch_size):
            utt_len = len(x_list[n])
            x[n, :utt_len] = x_list[n]
            lengths[n] = utt_len
            y[n, :utt_len] = y_list[n]
        yield x, y, lengths

### RNN

In [21]:
import tensorflow as tf
import numpy as np

C:\Users\moi\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [22]:
class BiLSTMModel():
    pass

##### Placeholers
* input_batch — sequences of words (shape =  [batch_size, sequence_len]);
* ground_truth_tags — sequences of tags (shape = [batch_size, sequence_len]);
* lengths — lengths of not padded sequences (shape= [batch_size]);
dropout_ph — dropout keep probability; 
learning_rate_ph 

In [23]:
def declare_placeholders(self):
    
    self.input_batch = tf.placeholder(dtype=tf.int32, shape=[None, None], name='input_batch') 
    self.ground_truth_tags = tf.placeholder(dtype=tf.int32, shape=[None, None], name='ground_truth_tags')
    self.lengths = tf.placeholder(dtype=tf.int32, shape=[None], name='lengths')
    self.dropout_ph = tf.placeholder_with_default(tf.cast(1.0, tf.float32), shape=[])
    self.learning_rate_ph = tf.placeholder_with_default(1e4, shape=[])

In [24]:
BiLSTMModel.__declare_placeholders = classmethod(declare_placeholders)

In [25]:
def build_layers(self, vocabulary_size, embedding_dim, n_hidden_rnn, n_tags):
    """Specifies bi-LSTM architecture and computes logits for inputs."""
    
    # embedding variable with dtype tf.float32
    initial_embedding_matrix = np.random.randn(vocabulary_size, embedding_dim) / np.sqrt(embedding_dim)
    embedding_matrix_variable = tf.Variable(initial_embedding_matrix, name='embeddings_matrix', dtype=tf.float32)
    
    # RNN cells with n_hidden_rnn number of units 
    # dropout 
    forward_cell = tf.nn.rnn_cell.DropoutWrapper(
        tf.nn.rnn_cell.BasicLSTMCell(num_units=n_hidden_rnn, forget_bias=3.0),
        input_keep_prob=self.dropout_ph,
        output_keep_prob=self.dropout_ph,
        state_keep_prob=self.dropout_ph
    )
    backward_cell = tf.nn.rnn_cell.DropoutWrapper(
        tf.nn.rnn_cell.BasicLSTMCell(num_units=n_hidden_rnn, forget_bias=3.0),
        input_keep_prob=self.dropout_ph,
        output_keep_prob=self.dropout_ph,
        state_keep_prob=self.dropout_ph
    )

    embeddings = tf.nn.embedding_lookup(embedding_matrix_variable, self.input_batch)
    
    # Bidirectional Dynamic RNN 
    # Shape: [batch_size, sequence_len, 2 * n_hidden_rnn]. 
    
    (rnn_output_fw, rnn_output_bw), _ = tf.nn.bidirectional_dynamic_rnn(
        cell_fw= forward_cell, cell_bw= backward_cell,
        dtype=tf.float32,
        inputs=embeddings,
        sequence_length=self.lengths
    )
    rnn_output = tf.concat([rnn_output_fw, rnn_output_bw], axis=2)

    # Dense layer.
    # Shape: [batch_size, sequence_len, n_tags].   
    self.logits = tf.layers.dense(rnn_output, n_tags, activation=None)

In [26]:
BiLSTMModel.__build_layers = classmethod(build_layers)

In [27]:
def compute_predictions(self):
    """Transforms logits to probabilities and finds the most probable tags."""
    
    # softmax function
    softmax_output = tf.nn.softmax(self.logits)
    
    # argmax (get the most probable tags)
    self.predictions = tf.argmax(softmax_output, axis=-1)

In [28]:
BiLSTMModel.__compute_predictions = classmethod(compute_predictions)

In [29]:
def compute_loss(self, n_tags, PAD_index):
    """cross-entopy loss"""
    
    ground_truth_tags_one_hot = tf.one_hot(self.ground_truth_tags, n_tags)
    loss_tensor = tf.nn.softmax_cross_entropy_with_logits(labels=ground_truth_tags_one_hot, logits=self.logits)
    
    # loss function which doesn't operate with <PAD> tokens (tf.reduce_mean)
    mask = tf.cast(tf.not_equal(loss_tensor, PAD_index), tf.float32)
    self.loss =  tf.reduce_mean(tf.reduce_sum(tf.multiply(loss_tensor, mask), axis=-1) / tf.reduce_sum(mask, axis=-1))

In [30]:
BiLSTMModel.__compute_loss = classmethod(compute_loss)

In [31]:
def perform_optimization(self):
    """Specifies the optimizer and train_op for the model."""
    
    # an optimizer (Adam)
    self.optimizer = tf.train.AdamOptimizer(self.learning_rate_ph)
    self.grads_and_vars = self.optimizer.compute_gradients(self.loss)
    
    # Gradient clipping (! vanishing/exploding)
    clip_norm = tf.cast(1.0, tf.float32)
    self.grads_and_vars = [(tf.clip_by_norm(grad, clip_norm), var) for grad, var in self.grads_and_vars]
    
    self.train_op = self.optimizer.apply_gradients(self.grads_and_vars)

In [32]:
BiLSTMModel.__perform_optimization = classmethod(perform_optimization)

In [33]:
def init_model(self, vocabulary_size, n_tags, embedding_dim, n_hidden_rnn, PAD_index):
    self.__declare_placeholders()
    self.__build_layers(vocabulary_size, embedding_dim, n_hidden_rnn, n_tags)
    self.__compute_predictions()
    self.__compute_loss(n_tags, PAD_index)
    self.__perform_optimization()

In [34]:
BiLSTMModel.__init__ = classmethod(init_model)

#### Train 

In [35]:
def train_on_batch(self, session, x_batch, y_batch, lengths, learning_rate, dropout_keep_probability):
    feed_dict = {self.input_batch: x_batch,
                 self.ground_truth_tags: y_batch,
                 self.learning_rate_ph: learning_rate,
                 self.dropout_ph: dropout_keep_probability,
                 self.lengths: lengths}
    
    session.run(self.train_op, feed_dict=feed_dict)

In [36]:
BiLSTMModel.train_on_batch = classmethod(train_on_batch)

In [37]:
def predict_for_batch(self, session, x_batch, lengths):

    predictions = session.run(self.predictions, feed_dict={self.input_batch:x_batch, self.lengths:lengths})
    return predictions

In [38]:
BiLSTMModel.predict_for_batch = classmethod(predict_for_batch)

#### Evaluation
Functions: 
* predict_tags: uses a model to get predictions
* eval_conll: calculates precision, recall and F1 for the results.

In [39]:
from evaluation import precision_recall_f1

In [40]:
def predict_tags(model, session, token_idxs_batch, lengths):
    """Performs predictions and transforms indices to tokens and tags."""
    
    tag_idxs_batch = model.predict_for_batch(session, token_idxs_batch, lengths)
    
    tags_batch, tokens_batch = [], []
    for tag_idxs, token_idxs in zip(tag_idxs_batch, token_idxs_batch):
        tags, tokens = [], []
        for tag_idx, token_idx in zip(tag_idxs, token_idxs):
            tags.append(idx2tag[tag_idx])
            tokens.append(idx2token[token_idx])
        tags_batch.append(tags)
        tokens_batch.append(tokens)
    return tags_batch, tokens_batch
    
    
def eval_conll(model, session, tokens, tags, short_report=True):
    
    y_true, y_pred = [], []
    for x_batch, y_batch, lengths in batches_generator(1, tokens, tags):
        tags_batch, tokens_batch = predict_tags(model, session, x_batch, lengths)
        if len(x_batch[0]) != len(tags_batch[0]):
            raise Exception("Incorrect length of prediction for the input, "
                            "expected length: %i, got: %i" % (len(x_batch[0]), len(tags_batch[0])))
        predicted_tags = []
        ground_truth_tags = []
        for gt_tag_idx, pred_tag, token in zip(y_batch[0], tags_batch[0], tokens_batch[0]): 
            if token != '<PAD>':
                ground_truth_tags.append(idx2tag[gt_tag_idx])
                predicted_tags.append(pred_tag)

        y_true.extend(ground_truth_tags + ['O'])
        y_pred.extend(predicted_tags + ['O'])
        
    results = precision_recall_f1(y_true, y_pred, print_results=True, short_report=short_report)
    return results

### Go! 

parameters:
* vocabulary_size — number of tokens;
* n_tags — number of tags;
* embedding_dim — dimension of embeddings, recommended: 200;
* n_hidden_rnn — size of hidden layers for RNN, recommended: 200;
* PAD_index —  index of the padding token (<PAD>).

**hyperparameters:** 
* batch_size
* epochs
* starting value of learning_rate
* learning_rate_decay: a square root of 2;
* dropout_keep_probability: try several values: 0.1, 0.5, 0.9.

In [41]:
tf.reset_default_graph()

model = BiLSTMModel(20505, 21, 200, 200, token2idx['<PAD>'])

batch_size = 32
n_epochs = 4
learning_rate = 0.005
learning_rate_decay = 1.414
dropout_keep_probability = 0.5

Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See tf.nn.softmax_cross_entropy_with_logits_v2.

Instructions for updating:
keep_dims is deprecated, use keepdims instead


In [42]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())

print('Start training... \n')
for epoch in range(n_epochs):
    # For each epoch evaluate the model on train and validation data
    print('-' * 20 + ' Epoch {} '.format(epoch+1) + 'of {} '.format(n_epochs) + '-' * 20)
    print('Train data evaluation:')
    eval_conll(model, sess, train_tokens, train_tags, short_report=True)
    print('Validation data evaluation:')
    eval_conll(model, sess, validation_tokens, validation_tags, short_report=True)
    
    # Train the model
    for x_batch, y_batch, lengths in batches_generator(batch_size, train_tokens, train_tags):
        model.train_on_batch(sess, x_batch, y_batch, lengths, learning_rate, dropout_keep_probability)
        
    # Decaying the learning rate
    learning_rate = learning_rate / learning_rate_decay
    
print('...training finished.')

Start training... 

-------------------- Epoch 1 of 4 --------------------
Train data evaluation:
processed 105778 tokens with 4489 phrases; found: 68770 phrases; correct: 166.

precision:  0.24%; recall:  3.70%; F1:  0.45

Validation data evaluation:
processed 12836 tokens with 537 phrases; found: 8281 phrases; correct: 14.

precision:  0.17%; recall:  2.61%; F1:  0.32

-------------------- Epoch 2 of 4 --------------------
Train data evaluation:
processed 105778 tokens with 4489 phrases; found: 2237 phrases; correct: 453.

precision:  20.25%; recall:  10.09%; F1:  13.47

Validation data evaluation:
processed 12836 tokens with 537 phrases; found: 192 phrases; correct: 48.

precision:  25.00%; recall:  8.94%; F1:  13.17

-------------------- Epoch 3 of 4 --------------------
Train data evaluation:
processed 105778 tokens with 4489 phrases; found: 4199 phrases; correct: 1886.

precision:  44.92%; recall:  42.01%; F1:  43.42

Validation data evaluation:
processed 12836 tokens with 537 ph

In [43]:
print('-' * 20 + ' Train set quality: ' + '-' * 20)
train_results = eval_conll(model, sess, train_tokens, train_tags, short_report=False)

print('-' * 20 + ' Validation set quality: ' + '-' * 20)
validation_results = eval_conll(model, sess, validation_tokens, validation_tags, short_report=False)

print('-' * 20 + ' Test set quality: ' + '-' * 20)
test_results = eval_conll(model, sess, test_tokens, test_tags, short_report=False)

-------------------- Train set quality: --------------------
processed 105778 tokens with 4489 phrases; found: 4637 phrases; correct: 3410.

precision:  73.54%; recall:  75.96%; F1:  74.73

	     company: precision:   81.29%; recall:   86.47%; F1:   83.80; predicted:   684

	    facility: precision:   63.86%; recall:   74.84%; F1:   68.91; predicted:   368

	     geo-loc: precision:   80.49%; recall:   92.37%; F1:   86.02; predicted:  1143

	       movie: precision:    0.00%; recall:    0.00%; F1:    0.00; predicted:     0

	 musicartist: precision:   50.66%; recall:   33.19%; F1:   40.10; predicted:   152

	       other: precision:   63.52%; recall:   79.13%; F1:   70.47; predicted:   943

	      person: precision:   82.96%; recall:   91.76%; F1:   87.14; predicted:   980

	     product: precision:   51.83%; recall:   49.06%; F1:   50.40; predicted:   301

	  sportsteam: precision:   81.82%; recall:   24.88%; F1:   38.16; predicted:    66

	      tvshow: precision:    0.00%; recall:  